In [12]:
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import to_categorical
import os
import cv2
from tqdm import tqdm
import random
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard

In [13]:
DATADIR = "/Users/ryanklapper/Desktop/2020/Data Science/search-bing-api/dataset/"

CATEGORIES = ["Bagpipes","Harmonica","Flute","Piano","Saxophone","Guitar",]
training_data = []
IMG_SIZE = 350
def train():
    for category in CATEGORIES:  # iterate each religion

        path = os.path.join(DATADIR,category)  # create path to different religions
        class_num = CATEGORIES.index(category)  # get the classification of each religion

        for img in tqdm(os.listdir(path)):  # iterate over each image in each religion
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:
                pass
            
train()

# Shuffling data so each religion is trained without bias
random.shuffle(training_data)

X = []
y = []
pictures = []
for music_features,music_label in training_data:
    X.append(music_features)
    y.append(music_label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)

100%|██████████| 201/201 [00:04<00:00, 48.86it/s]


In [14]:
pickle_save_X = open("X.pickle","wb")
pickle.dump(X, pickle_save_X)
pickle_save_X.close()

pickle_save_y = open("y.pickle","wb")
pickle.dump(y, pickle_save_y)
pickle_save_y.close()

In [15]:
NAME = "Is-Music-Relevant-CNN"

pickle_load_X = open("X.pickle","rb")
X = pickle.load(pickle_load_X)

pickle_load_y = open("y.pickle","rb")
y = pickle.load(pickle_load_y)

X = X/255.0

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=15,
    width_shift_range=0.14,
    height_shift_range=0.14,
    horizontal_flip=True)

datagen.fit(X)

model = Sequential()

model.add(Conv2D(30, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(30, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(6))
model.add(Activation("sigmoid"))

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Go through 10 samples at a time
# Train 70% , Test 30%
# Iterate 3 times (epochs = 3)
epochs = 4
model.fit_generator(datagen.flow(X, y, batch_size=10),
                    steps_per_epoch=len(X) / 10, epochs=epochs,callbacks=[tensorboard])



Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 114.3 steps
Epoch 1/4
115/114 [==============================] - 56s 490ms/step - loss: 2.9485 - accuracy: 0.2485
Epoch 2/4
115/114 [==============================] - 58s 507ms/step - loss: 1.6237 - accuracy: 0.3351
Epoch 3/4
115/114 [==============================] - 57s 493ms/step - loss: 1.4893 - accuracy: 0.3788
Epoch 4/4
115/114 [==============================] - 55s 481ms/step - loss: 1.3110 - accuracy: 0.4864
